# Automatic Differentiation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChemAI-Lab/AI4Chem/blob/main/website/modules/03-automatic_differentiation.ipynb)
   

**References:**
1. **Automatic Differentiation in Machine Learning: a Survey** [Paper PDF](https://www.jmlr.org/papers/volume18/17-468/17-468.pdf)
1. **Chapters 5**: [Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf), C. M. Bishop.
<!-- 2. **Chapter 2**: [Machine Learning in Quantum Sciences](https://arxiv.org/pdf/2204.04198) UPDATE THIS! -->
3. **Chapter 7**: [Probabilistic Machine Learning: An Introduction, K. P. Murphy.](https://probml.github.io/pml-book/book1.html)

